## Tiến Hành thu thập dữ liệu

---

In [345]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests as r
import csv
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from datetime import date

#### Dùng requests để lấy data từ trang ```https://www.worldometers.info/coronavirus```

In [275]:
headers = {'User-Agent': 'Mozilla/5.0'}
page = r.get("https://www.worldometers.info/coronavirus", headers)
soup  = BeautifulSoup(page.text, 'html.parser')

#### Hàm lấy tên của cột

In [338]:
def getTitle():
    title = soup.find_all("thead")
    # title
    result_title = []
    result_title.append("Date")
    for t in title:
        result = t.find_all("th")
        for r in result[1:]:
            temp = r.text
            temp = temp.replace("\n", "")
            result_title.append(temp)
        break


    return result_title

#### Hàm lấy dữ liệu từ now, yesterday, 2 days ago

In [339]:
def getData(_type):
    result = []
    # now
    if (_type == 1):
        data = soup.find_all("table", {"id": "main_table_countries_today"})
    # yesterday
    elif (_type == 2):
        data = soup.find_all("table", {"id": "main_table_countries_yesterday"})
    # 2 days ago
    else:
        data = soup.find_all("table", {"id": "main_table_countries_yesterday2"})
    
    for D in data:
        data = D.find_all("tbody")
        for d in data:
            d.find_all("tr")
            for value in d:
                try:
                    temp = value["style"]
                    if (temp != "display: none"):
                        result.append(value)
                except:
                    None
            break    
        break
        
    return result
    

#### Thực hiện các thao tác lấy dữ liệu 

In [340]:
# lay tieu de cua cot
result_title = getTitle()

# lấy thông tin của ngày hôm nay
result_now = getData(1)

# lấy thông tin của ngày hôm qua
result_yesterday = getData(2)

# lấy thông tin của 2 ngày trước
result_twoDaysAgo = getData(3)

In [364]:
with open("Data\Covid.csv", "a", encoding="UTF8") as f_object:
    writer_object = csv.writer(f_object)
    # write data
    for r in result_yesterday:
        temp = []
        temp.append("30/11/2021")
        for i in r.find_all("td")[1:]:
            temp.append(i.text)
        
        #write
        writer_object.writerow(temp)
    f_object.close()

In [365]:
data_covid = pd.read_csv("Data\Covid.csv")

In [366]:
data_covid

,Date,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",...,TotalTests,Tests/1M pop,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl,New Cases/1M pop,New Deaths/1M pop,Active Cases/1M pop
0,1/12/2021,USA,"49,428,913",NaN,"803,045",NaN,"39,205,902",NaN,"9,419,966","13,201",...,"753,993,369","2,259,195","333,744,223",North America,7,416,0.0,NaN,NaN,"28,225"
1,1/12/2021,India,"34,596,776",NaN,"469,247",NaN,"34,028,506","+10,207","99,023","8,944",...,"642,412,315","459,141","1,399,161,383",Asia,40,"2,982",2.0,NaN,NaN,71
2,1/12/2021,Brazil,"22,094,459",NaN,"614,754",NaN,"21,321,631",NaN,"158,074","8,318",...,"63,776,166","297,057","214,693,373",South America,10,349,3.0,NaN,NaN,736
3,1/12/2021,UK,"10,228,772",NaN,"144,969",NaN,"9,062,561",NaN,"1,021,242",916,...,"362,531,048","5,300,958","68,389,728",Europe,7,472,0.0,NaN,NaN,"14,933"
4,1/12/2021,Russia,"9,636,881",NaN,"275,193",NaN,"8,329,253",NaN,"1,032,435","2,300",...,"224,900,000","1,540,170","146,022,882",Europe,15,531,1.0,NaN,NaN,"7,070"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,30/11/2021,Marshall Islands,4,NaN,,NaN,4,NaN,0,NaN,...,NaN,NaN,"59,754",Australia/Oceania,"14,939",NaN,NaN,NaN,NaN,NaN
668,30/11/2021,Samoa,3,NaN,,NaN,3,NaN,0,NaN,...,NaN,NaN,"200,276",Australia/Oceania,"66,759",NaN,NaN,NaN,NaN,NaN
669,30/11/2021,Saint Helena,2,NaN,,NaN,2,NaN,0,NaN,...,NaN,NaN,"6,103",Africa,"3,052",NaN,NaN,NaN,NaN,NaN
670,30/11/2021,Micronesia,1,NaN,,NaN,1,NaN,0,NaN,...,NaN,NaN,"116,725",Australia/Oceania,"116,725",NaN,NaN,NaN,NaN,NaN


In [373]:
sort_df = data_covid.sort_values(by=["Country,Other"])

In [375]:
sort_df[0:12]

,Date,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",...,TotalTests,Tests/1M pop,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl,New Cases/1M pop,New Deaths/1M pop,Active Cases/1M pop
101,1/12/2021,Afghanistan,"157,289",NaN,"7,308",NaN,"140,549",NaN,"9,432","1,124",...,"792,156","19,723","40,164,434",Asia,255,"5,496",51.0,NaN,NaN,235
325,29/11/2021,Afghanistan,"157,261",+33,"7,308",NaN,"140,530",+53,"9,423","1,124",...,"791,514","19,707","40,164,434",Asia,255,"5,496",51.0,0.8,NaN,235
550,30/11/2021,Afghanistan,"157,289",+28,"7,308",NaN,"140,549",+19,"9,432","1,124",...,"792,156","19,723","40,164,434",Asia,255,"5,496",51.0,0.7,NaN,235
95,1/12/2021,Albania,"199,945",NaN,"3,096",NaN,"189,784",NaN,"7,065",23,...,"1,394,136","485,194","2,873,357",Europe,14,928,2.0,NaN,NaN,"2,459"
320,29/11/2021,Albania,"199,750",+195,"3,092",+3,"189,227",+183,"7,431",23,...,"1,392,736","484,707","2,873,357",Europe,14,929,2.0,68,1.0,"2,586"
544,30/11/2021,Albania,"199,945",+195,"3,096",+4,"189,784",+557,"7,065",23,...,"1,394,136","485,194","2,873,357",Europe,14,928,2.0,68,1.0,"2,459"
93,1/12/2021,Algeria,"210,531",NaN,"6,071",NaN,"144,450",NaN,"60,010",14,...,"230,861","5,134","44,968,076",Africa,214,"7,407",195.0,NaN,NaN,"1,335"
317,29/11/2021,Algeria,"210,344",+192,"6,064",+6,"144,295",+135,"59,985",17,...,"230,861","5,134","44,968,076",Africa,214,"7,416",195.0,4,0.1,"1,334"
541,30/11/2021,Algeria,"210,531",+187,"6,071",+7,"144,450",+155,"60,010",14,...,"230,861","5,134","44,968,076",Africa,214,"7,407",195.0,4,0.2,"1,335"
385,29/11/2021,Andorra,"17,115",+403,131,NaN,"15,903",+133,"1,081",4,...,"193,595","2,499,935","77,440",Europe,5,591,0.0,"5,204",NaN,"13,959"
